## Robust Production - Inventory

The production inventory problem is one that addresses uncertainty in inventory management scenarios. The objective is to minimize the costs associated with inventory management while ensuring that customer demand is met consistently over a planning horizon. Since the demand is uncertain and varies within a specific bound, this problem can be classified as a robust optimization problem.

We can consider a single product inventory system which is comprised of a warehouse and $I$ factories. The planning horizon is $T$ periods. At period $t$:
- $d_t$ is the demand of the product that is uncertain
- $v(t)$ is the amount of product in the warehouse at the beginning of a period
- $p_i(t)$ is the amount of product to be produced during period $t$ by factory $i$ and is used to satisfy the demand of the period
- $P_i(t)$ is the maximal production capacity of factory $i$ in time period $t$
- $c_i(t)$ is the cost of producing a unit of product at factory $i$ in time period $t$
- $Q_i(t)$ is the maximal cumulative production capacity of factory $i$ in time period $t$
- $V_{min}$ and $V_{max}$ are the minimal and maximal storage capacity of the warehouse, respectively

With this information, the robust counterpart to the problem can be modeled into the following linear program:

$$
\begin{aligned}
& \text{minimize} \quad  F \\
& \text{subject to} \\
& \vec{c(t)}^T\vec{p(t)} \leq F,\\
& 0 \leq p_i(t) \leq P_i(t),\\
& 1^T\vec{p(t)} \leq Q_i,\\
& V_{min} \leq v(1) + 1^T\vec{p} - 1^T\vec{d} \leq V_{max}\\

\end{aligned}
$$


To solve an example of this problem, we first import the required packages and generate the data. 

In [30]:
import numpy as np 
import cvxpy as cp 
import lropt
import warnings
from scipy.sparse import SparseEfficiencyWarning
warnings.filterwarnings('ignore', category=UserWarning, module='cvxpy')
warnings.filterwarnings('ignore', category=SparseEfficiencyWarning)

In our example, we consider $I$ = 3 factories producing a product in one warehouse. The time horizon $T$ is 24 periods. The  maximal production capacity of each one of the factories at each two-weeks period is $P_i(t) = 567$, and the integral production capacity of each one of the factories for a year is $Q = 13600$. The inventory at the warehouse should not be less then $500$ units, and cannot exceed $2000$ units. Initially, the inventory sits at $500$ units.

In [31]:
np.random.seed(1)
T = 24 #number of periods
I = 3 #number of factories
V_MIN = 500
V_MAX = 2000
HALF = 0.5
SEASONAL_MULTIPLIER = 1000
INTEGRAL_CAPACITY = 13600
RHO = 18284138416584168
V_INITIAL = 500
MAX_CAPACITY = 567
PROPORTION = 0.2

P = np.full((I, T), MAX_CAPACITY)
Q = [INTEGRAL_CAPACITY]*I

F = cp.Variable(nonneg = True)
p = cp.Variable((I,T), nonneg = True)


The production cost for a factory $i$ at a period $t$ is given by:
$$
\begin{align*}
c_i(t) =  \alpha_i \left(1 - \frac{1}{2} \sin \left(\frac{\pi (t-1)}{12}\right)\right), \quad t  \in [1, 2, ... ,24] \\
\alpha_i \in [1, 1.5, 2]
\end{align*}
$$

In [32]:
c = [] 
alphas = np.array([1, 1.5, 2])

for t in range (1, T+1):
    c.append((alphas * (1 + HALF*np.sin(np.pi*(t-1)/(T*HALF)))).flatten())
c = np.array(c).T


In the robust optimization context, the demand is considered uncertain and seasonal, reaching its peak in winter. Specifically, the demand follows a seasonal pattern represented by:
$$
\begin{align*}
d^*(t) =  1000\left(1 - \frac{1}{2} \sin \left(\frac{\pi (t-1)}{12}\right)\right), \quad  t  \in [1, 2, ... ,24] \\
\end{align*}
$$
To handle this uncertainty, we adopt an ellipsoidal uncertainty model. This approach allows us to define bounds for the uncertain set using parameters $c$ and $d$. Within this frameework, $F, p$ and $v$ are variables in CVXPY.

In this setup, robust optimization ensures that the decisions made are resilient against variations in demand within an uncertainty level of $20\%$. This robustness is achieved by optimizing over a range of possible demand scenarios rather than relying on a single forecast.

In [33]:
t_values = np.arange(1, T + 1)
d_star = SEASONAL_MULTIPLIER * (1 + HALF * np.sin(np.pi * (t_values - 1) / (T * HALF)))

# Construct lhs
eye_matrix = np.eye(T)
neg_eye_matrix = -np.eye(T)
lhs = np.concatenate((eye_matrix, neg_eye_matrix), axis=0)

# Construct rhs
rhs_upper = (1 + PROPORTION) * d_star
rhs_lower = (-1 + PROPORTION) * d_star
rhs = np.hstack((rhs_upper, rhs_lower))

# Uncertain parameter definition
d = lropt.UncertainParameter(T, uncertainty_set=lropt.Norm(rho=RHO, c=lhs, d=rhs))

Next, as mentioned earlier, we will define all the constraints of the problem in a list.

In [34]:
constraints = [
    cp.sum(cp.multiply(c, p)) <= F,
    p <= P,
    cp.sum(p) <= Q[0],
    V_MIN <= cp.sum(p) - cp.sum(d) + V_INITIAL,
    cp.sum(p) - cp.sum(d) + V_INITIAL <= V_MAX
]

Finally, we can define the objective and solve the problem. 

In [35]:
objective = cp.Minimize(F)
prob = lropt.RobustProblem(objective, constraints)
prob.solve()
sol = F.value
print(f"The robust optimal value is {sol}")

The robust optimal value is 2.3576801416664123e-09
